In [8]:
import os

In [3]:
%pwd

'd:\\Sanket Nikam\\Programs\\Jupyter Notebook\\Data Science\\Projects\\ShipIt-ML\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Sanket Nikam\\Programs\\Jupyter Notebook\\Data Science\\Projects\\ShipIt-ML'

In [2]:
import mlflow

print("Current MLflow Tracking URI:", mlflow.get_tracking_uri())

Current MLflow Tracking URI: https://dagshub.com/SannketNikam/ShipIt-ML.mlflow


In [3]:
dagshub_tracking_uri = "https://dagshub.com/SannketNikam/ShipIt-ML.mlflow"
mlflow.set_tracking_uri(dagshub_tracking_uri)

print("New Tracking URI:", mlflow.get_tracking_uri())  # Confirm it's set correctly

New Tracking URI: https://dagshub.com/SannketNikam/ShipIt-ML.mlflow


In [ ]:
!mlflow ui --backend-store-uri "https://dagshub.com/SannketNikam/ShipIt-ML.mlflow"

2025/03/31 22:06:27 ERROR mlflow.cli: Error initializing backend store
2025/03/31 22:06:27 ERROR mlflow.cli:  Model registry functionality is unavailable; got unsupported URI 'https://dagshub.com/SannketNikam/ShipIt-ML.mlflow' for model registry data storage. Supported URI schemes are: ['', 'file', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.
Traceback (most recent call last):
  File "C:\Users\Sanket\anaconda3\envs\mlops\lib\site-packages\mlflow\tracking\registry.py", line 77, in get_store_builder
    store_builder = self._registry[scheme]
KeyError: 'https'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Sanket\anaconda3\envs\mlops\lib\site-packages\mlflow\cli.py", line 392, in server
    initialize_backend_stores(backend_store_uri, registry_store_uri, default_artifact

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SannketNikam/ShipIt-ML.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SannketNikam"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0127ae428cdaeb2bd2444a8e5059355946d371c7"

In [6]:
import mlflow

# Set remote tracking URI (DagsHub)
mlflow.set_tracking_uri("https://dagshub.com/SannketNikam/ShipIt-ML.mlflow")

with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_metric("accuracy", 0.92)

print("Run logged! Check DagsHub UI for details.")


Run logged! Check DagsHub UI for details.


In [1]:
import dagshub
dagshub.init(repo_owner='SannketNikam', repo_name='ShipIt-ML', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as SannketNikam

Initialized MLflow to track repo "SannketNikam/ShipIt-ML"

Repository SannketNikam/ShipIt-ML initialized!

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/SannketNikam/ShipIt-ML.mlflow"
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving Metrics as local
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path = Path(self.config.metric_file_name), data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                """
                Register the model
                There are other ways to use the Model Registry, which depends on the use case
                Please refer to the doc for more information:
                https://mlflow.org/docs/latest/model-registry.html#api-workflow
                """
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-29 23:07:17,930: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-03-29 23:07:17,931: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-03-29 23:07:17,940: INFO: common: Yaml file: schema.yaml loaded successfully]
[2025-03-29 23:07:17,940: INFO: common: Created directory at: artifacts/]
[2025-03-29 23:07:17,946: INFO: common: Created directory at: artifacts/model_evaluation]


[2025-03-29 23:07:18,494: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Sanket\anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2025/03/29 23:07:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 3
Created version '3' of model 'ElasticNetModel'.
